In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 2.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633967 sha256=e88eb09a4dc47c6bd6f798c80da62fa3583e8f7ade7d28861254703cba84bb7c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [32]:
# imports
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD # for collabrotative filtering and matrix factorization
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np

In [138]:
sd = pd. read_csv('Steam Dataset.csv')
sd.head()

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description
0,285421,3487,620,Portal 2,0.987930,https://store.steampowered.com/app/620,"<div class=""game_area_description"" id=""game_ar..."
1,102805,1640,400,Portal,0.984298,https://store.steampowered.com/app/400,"<div class=""game_area_description"" id=""game_ar..."
2,439665,8619,413150,Stardew Valley,0.980773,https://store.steampowered.com/app/413150,"<div class=""game_area_description"" id=""game_ar..."
3,476762,9701,431960,Wallpaper Engine,0.980058,https://store.steampowered.com/app/431960,"<div class=""game_area_description"" id=""game_ar..."
4,910629,19342,105600,Terraria,0.979202,https://store.steampowered.com/app/105600,"<div class=""game_area_description"" id=""game_ar..."


**Through this code, I extract a column " Game_description" from the HTML code**

In [139]:
import re
import string

current_count = 0
for i in range(0,100):
    sd['Game_description'][i] = re.sub('<.*?>', ' ', sd['Game_description'][i])
    sd['Game_description'][i] = re.sub('\\n.*\\n', ' ', sd['Game_description'][i])
    sd['Game_description'][i] = sd['Game_description'][i].replace("\t","")
    sd['Game_description'][i] = sd['Game_description'][i].translate(sd['Game_description'][i].maketrans(' ',' ',string.punctuation))


    current_count+=1


I use   "fillna"  method to fill values in Game_description column

In [140]:

sd['Game_description'] = sd['Game_description'].fillna('[]')

display  Game_description column to check 

In [141]:

 sd['Game_description']

0       Portal 2 draws from the awardwinning formula...
1        Portal™ is a new single player game from Va...
2       Stardew Valley is an openended countrylife R...
3       Wallpaper Engine enables you to use live wal...
4       Dig Fight Explore Build  The very world is a...
                            ...                        
95       LAND LOOT SURVIVE  Play PUBG BATTLEGROUNDS ...
96      Z1 Battle Royale is a Free to Play fastpaced...
97        TEST SERVER   Z1 Battle Royale is a Free t...
98      NOTE All multiplayer servers for NBA 2K20 wi...
99       WELCOME TO 2042 Battlefield™ 2042 is a firs...
Name: Game_description, Length: 100, dtype: object

here i check the Rate_counts above the 95th percentile 


In [143]:

Rate_counts = sd[sd['Positive'].notnull()]['Positive'].astype('float64')
Rate_averages = sd[sd['Positive_Rate'].notnull()]['Positive_Rate'].astype('float64')
C = Rate_averages.mean() #average of  Positive  .
C

0.8498376658669073

In [144]:
m = Rate_counts.quantile(0.95)
m

907781.85

getting data matching the above criteria (qualified game)


In [145]:
qualified = sd[(sd['Positive'] >= m) & (sd['Positive'].notnull()) & (sd['Positive'].notnull())][['Positive', 'Negative', 'Game_name', 'Positive_Rate', 'links','Game_description']]
qualified['Positive'] = qualified['Positive'].astype('float64')
qualified['Positive_Rate'] = qualified['Positive_Rate'].astype('float64')
qualified.shape

(5, 6)

In [146]:
def weighted_rating(x):


  v = x['Positive']
  R = x['Positive_Rate']
  return (v/(v+m) * R) + (m/(m+v) * C)



In [147]:
qualified['Rate'] = qualified.apply(weighted_rating, axis=1)

top game

In [148]:
qualified.sort_values('Rate', ascending=False).head(15)

,Positive,Negative,Game_name,Positive_Rate,links,Game_description,Rate
4,910629.0,19342,Terraria,0.979202,https://store.steampowered.com/app/105600,Dig Fight Explore Build The very world is a...,0.914621
48,5549226.0,741941,Counter-Strike: Global Offensive,0.882066,https://store.steampowered.com/app/730,CounterStrike Global Offensive CS GO expands...,0.877535
62,1110207.0,205439,Grand Theft Auto V,0.843849,https://store.steampowered.com/app/271590,When a young street hustler a retired bank r...,0.846543
63,1418211.0,279704,Dota 2,0.835266,https://store.steampowered.com/app/570,The mostplayed game on Steam Every day mil...,0.840953
95,1108250.0,875814,PUBG: BATTLEGROUNDS,0.558576,https://store.steampowered.com/app/578080,LAND LOOT SURVIVE Play PUBG BATTLEGROUNDS ...,0.689726


i construct function to builds recommendation charts for particular game with conditions to the 85th percentile instead of 95.

In [149]:
#i doing this to have each unique game name records separately, so that we could build recommendations for each game name 

s = sd.apply(lambda x: pd.Series(x['Game_name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'game'
gen_sd = sd.drop('Game_name', axis=1).join(s)

In [69]:


def build_chart(game, percentile=0.85):
    df = gen_sd[gen_sd['game'] == game]
    Positives = df[df['Positive'].notnull()]['Positive'].astype('float64')
    Positive_Rates = df[df['Positive_Rate'].notnull()]['Positive_Rate'].astype('float64')
    C = Positive_Rates.mean()
    m = Positives.quantile(percentile)
    
    qualified = df[(df['Positive'] >= m) & (df['Positive'].notnull()) & (df['Positive_Rate'].notnull())][['Positive', 'Negative', 'game', 'Positive_Rate', 'links']]
    qualified['Positive'] = qualified['Positive'].astype('float64')
    qualified['Positive_Rate'] = qualified['Positive_Rate'].astype('float64')
    
    qualified['Rate'] = qualified.apply(lambda x: (x['Positive']/(x['Positive']+m) * x['Positive_Rate']) + (m/(m+x['Positive']) * C), axis=1)
    qualified = qualified.sort_values('Rate', ascending=False).head(250)
    
    return qualified



In [151]:
build_chart('Terraria').head()

,Positive,Negative,game,Positive_Rate,links,Rate
4,910629.0,19342,Terraria,0.979202,https://store.steampowered.com/app/105600,0.979202


In [152]:

build_chart('Portal').head()

,Positive,Negative,game,Positive_Rate,links,Rate
1,102805.0,1640,Portal,0.984298,https://store.steampowered.com/app/400,0.984298


In [74]:
links_small = pd.read_csv('//content/Steam Dataset.csv')
links_small = links_small[links_small['Game_id'].notnull()]['Game_id'].astype('int')


In [76]:

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

sd['Game_id'] = sd['Game_id'].apply(convert_int)
print(sd.Game_id.isna().sum())
sd = sd.dropna(axis=0, subset=['Game_id'])
print(sd.Game_id.isna().sum())



0
0


In [79]:
smd = sd[sd['Game_id'].isin(links_small)]
smd.shape


(100, 7)

In [87]:
# we will combine the Positive_Rate with Game_description and  Game_name to create recommendation 
smd['Positive_Rate'] = smd['Positive_Rate'].astype(str)
smd['recommendation'] = "RATE: " + smd['Positive_Rate'] +" description :" + smd['Game_description'] + smd['Game_name']
smd['recommendation'] = smd['recommendation'].fillna('')
smd.head()

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description,recommendation
0,285421,3487,620,Portal 2,0.9879304138341616,https://store.steampowered.com/app/620,Portal 2 draws from the awardwinning formula...,RATE: 0.9879304138341616 description : Portal...
1,102805,1640,400,Portal,0.9842979558619368,https://store.steampowered.com/app/400,Portal™ is a new single player game from Va...,RATE: 0.9842979558619368 description : Porta...
2,439665,8619,413150,Stardew Valley,0.9807733490376636,https://store.steampowered.com/app/413150,Stardew Valley is an openended countrylife R...,RATE: 0.9807733490376636 description : Starde...
3,476762,9701,431960,Wallpaper Engine,0.9800580928045916,https://store.steampowered.com/app/431960,Wallpaper Engine enables you to use live wal...,RATE: 0.9800580928045916 description : Wallpa...
4,910629,19342,105600,Terraria,0.97920150198232,https://store.steampowered.com/app/105600,Dig Fight Explore Build The very world is a...,RATE: 0.97920150198232 description : Dig Figh...


In [88]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['recommendation'])

In [89]:
tfidf_matrix.shape

(100, 20520)

In [90]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [91]:
print(cosine_sim.shape)
print(cosine_sim[0])

(100, 100)
[1.00000000e+00 1.72164666e-01 2.75949919e-02 1.55862601e-02
 1.65261776e-02 2.15409486e-02 1.05628968e-02 1.75828718e-02
 5.08222068e-02 1.56145969e-02 2.23261862e-02 4.91588778e-02
 1.88130043e-02 1.15614691e-02 1.49547115e-02 2.51017371e-02
 2.25383983e-02 1.32387137e-02 1.71440070e-02 3.79563114e-02
 3.49562046e-02 7.55738086e-03 2.29027660e-02 1.84586768e-02
 1.36808033e-02 8.82375035e-03 3.83731920e-02 1.34552377e-02
 1.62172062e-02 1.77400018e-02 7.27719148e-03 6.13127237e-02
 2.72370087e-02 1.44989036e-02 1.15830040e-02 4.05494316e-02
 2.44898100e-02 6.96160372e-04 3.96995571e-02 1.62244342e-02
 2.08501978e-02 3.65532244e-02 1.18978185e-02 1.49589175e-02
 2.60778690e-02 1.80807872e-02 6.04139269e-03 1.95106328e-02
 2.40899824e-02 1.40374213e-02 3.19582988e-02 1.31171917e-02
 1.84668186e-02 7.36090980e-03 2.21749242e-02 1.70955273e-02
 2.37349991e-02 1.43965326e-02 1.67558763e-02 1.80047336e-02
 2.39742463e-02 2.46422696e-02 4.17580613e-02 2.19103305e-02
 2.19033850e-

In [92]:
smd = smd.reset_index()
GameName = smd['Game_name']
indices = pd.Series(smd.index, index=smd['Game_name'])

In [156]:
def get_recommendations(Game_name):
    idx = indices[Game_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] 
    game_indices = [i[0] for i in sim_scores]
    return GameName.iloc[game_indices]

In [101]:
get_recommendations('Portal').head(10)

0                             Portal 2
8                        Left 4 Dead 2
38      Counter-Strike: Condition Zero
31      Call of Duty: Modern Warfare 2
19           Half-Life: Opposing Force
48    Counter-Strike: Global Offensive
26                       Borderlands 2
52                 The Tiny Bang Story
2                       Stardew Valley
16          Sid Meier's Civilization V
Name: Game_name, dtype: object

In [104]:

get_recommendations('Wallpaper Engine').head(10)

68                           ARK: Survival Evolved
41                                          Arma 3
32    Guacamelee! Super Turbo Championship Edition
29                                Cities: Skylines
61                                      Brawlhalla
36                                        Unturned
6                           Euro Truck Simulator 2
20                                     Dying Light
24                                Human: Fall Flat
23                     The Elder Scrolls V: Skyrim
Name: Game_name, dtype: object

In [116]:

reader = Reader()
con=0

ratings = sd
ratings['UserId'] = np.random.randint(1,30000, ratings.shape[0])
ratings.head()

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description,UserId
0,285421,3487,620,Portal 2,0.987930,https://store.steampowered.com/app/620,Portal 2 draws from the awardwinning formula...,2722
1,102805,1640,400,Portal,0.984298,https://store.steampowered.com/app/400,Portal™ is a new single player game from Va...,23049
2,439665,8619,413150,Stardew Valley,0.980773,https://store.steampowered.com/app/413150,Stardew Valley is an openended countrylife R...,3597
3,476762,9701,431960,Wallpaper Engine,0.980058,https://store.steampowered.com/app/431960,Wallpaper Engine enables you to use live wal...,29518
4,910629,19342,105600,Terraria,0.979202,https://store.steampowered.com/app/105600,Dig Fight Explore Build The very world is a...,24867


In [117]:
sum(ratings['UserId'].duplicated())


0

In [118]:
data = Dataset.load_from_df(ratings[['Game_id', 'Positive_Rate','UserId']], reader)

In [119]:
ratings['Positive_Rate'] = ratings['Positive_Rate'].astype('float64')

In [120]:
svd = SVD()
cross_validate(svd, data,cv=5, measures=['RMSE', 'MAE'])

{'test_rmse': array([18585.94495714, 16545.68961694, 14934.78330944, 15827.59129495,
        16062.5048358 ]),
 'test_mae': array([16179.25, 14099.  , 12976.3 , 13876.5 , 14540.5 ]),
 'fit_time': (0.0209805965423584,
  0.025957822799682617,
  0.012965679168701172,
  0.025681734085083008,
  0.009088516235351562),
 'test_time': (0.0003986358642578125,
  0.00037670135498046875,
  0.00041222572326660156,
  0.00044035911560058594,
  0.0003948211669921875)}

In [121]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [124]:

ratings_1 = ratings[ratings['UserId'] == 23049]
ratings_1.rename(columns={'Game_id':'id'},inplace=True)
ratings_1.dropna(inplace=True)
ratings_1

,Positive,Negative,id,Game_name,Positive_Rate,links,Game_description,UserId
1,102805,1640,400,Portal,0.984298,https://store.steampowered.com/app/400,Portal™ is a new single player game from Va...,23049


In [132]:
ratings_pred = []
for index, row in ratings_1.iterrows():
  pred = svd.predict(row['id'],row['id'])
  ratings_pred.append(pred[3])
ratings_1['predicted_rating']= ratings_pred

In [133]:
ratings_1['Positive_Rate'] = ratings_1['Positive_Rate'].astype('float64')
error = np.abs(ratings_1['predicted_rating']-ratings_1['Positive_Rate']) 
error_perc = (error/ratings_1['Positive_Rate'])*100
ratings_1['rating_error'] = error
ratings_1['rating_error_percentage']= error_perc
ratings_1

,Positive,Negative,id,Game_name,Positive_Rate,links,Game_description,UserId,predicted_rating,rating_error,rating_error_percentage
1,102805,1640,400,Portal,0.984298,https://store.steampowered.com/app/400,Portal™ is a new single player game from Va...,23049,5,4.015702,407.976266


let's get error avregae for this user 

In [134]:
print('error aevrage:', ratings_1['rating_error'].mean())

error aevrage: 4.015702044138063


In [155]:
x= svd.predict(1, 8844)
x[3]

5